In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1627363,2021-09-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1627364,2021-09-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1627365,2021-09-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1627366,2021-09-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
42595,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42597,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42599,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42601,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
42603,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
992993,2021-09-05,Arkansas,Arkansas,5001,3186,54.00,5000,Arkansas,AR,Arkansas,State,3017804
992995,2021-09-06,Arkansas,Arkansas,5001,3189,54.00,5000,Arkansas,AR,Arkansas,State,3017804
992997,2021-09-07,Arkansas,Arkansas,5001,3191,54.00,5000,Arkansas,AR,Arkansas,State,3017804
992999,2021-09-08,Arkansas,Arkansas,5001,3195,54.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1627363,2021-09-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1627364,2021-09-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1627365,2021-09-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1627366,2021-09-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-09-09') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
597,2021-09-09,Snohomish,Washington,53510,658.00,53061,WA,County,822083,6509.08,80.04
1192,2021-09-09,Cook,Illinois,601238,11396.00,17031,IL,County,5150233,11674.00,221.27
1786,2021-09-09,Orange,California,309576,5295.00,6059,CA,County,3175692,9748.30,166.74
2379,2021-09-09,Maricopa,Arizona,659003,10992.00,4013,AZ,County,4485414,14692.13,245.06
2972,2021-09-09,Los Angeles,California,1426515,25563.00,6037,CA,County,10039107,14209.58,254.63
...,...,...,...,...,...,...,...,...,...,...,...
1626163,2021-09-09,Wheeler,Oregon,62,1.00,41069,OR,County,1332,4654.65,75.08
1626495,2021-09-09,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1626796,2021-09-09,Esmeralda,Nevada,43,0.00,32009,NV,County,873,4925.54,0.00
1627093,2021-09-09,Loving,Texas,7,0.00,48301,TX,County,169,4142.01,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
597,2021-09-09,Snohomish,Washington,53510,658.00,53061,WA,County,822083,6509.08,80.04,80.04,6509.08
1192,2021-09-09,Cook,Illinois,601238,11396.00,17031,IL,County,5150233,11674.00,221.27,221.27,11674.00
1786,2021-09-09,Orange,California,309576,5295.00,6059,CA,County,3175692,9748.30,166.74,166.74,9748.30
2379,2021-09-09,Maricopa,Arizona,659003,10992.00,4013,AZ,County,4485414,14692.13,245.06,245.06,14692.13
2972,2021-09-09,Los Angeles,California,1426515,25563.00,6037,CA,County,10039107,14209.58,254.63,254.63,14209.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626163,2021-09-09,Wheeler,Oregon,62,1.00,41069,OR,County,1332,4654.65,75.08,75.08,4654.65
1626495,2021-09-09,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1626796,2021-09-09,Esmeralda,Nevada,43,0.00,32009,NV,County,873,4925.54,0.00,0.00,4925.54
1627093,2021-09-09,Loving,Texas,7,0.00,48301,TX,County,169,4142.01,0.00,0.00,4142.01


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1038578,2021-09-09,Hale,Alabama,2786,81.00,1065,AL,County,14651,19015.77,552.86,552.86,19015.77,1
1131073,2021-09-09,Clarke,Alabama,4382,68.00,1025,AL,County,23622,18550.50,287.87,287.87,18550.50,2
719205,2021-09-09,Etowah,Alabama,17962,400.00,1055,AL,County,102268,17563.66,391.13,391.13,17563.66,3
603036,2021-09-09,Franklin,Alabama,5490,91.00,1059,AL,County,31362,17505.26,290.16,290.16,17505.26,4
720273,2021-09-09,Marshall,Alabama,16531,249.00,1095,AL,County,96774,17082.07,257.30,257.30,17082.07,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873942,2021-09-09,Johnson,Wyoming,971,13.00,56019,WY,County,8445,11497.93,153.94,153.94,11497.93,19
1374401,2021-09-09,Niobrara,Wyoming,243,3.00,56027,WY,County,2356,10314.09,127.33,127.33,10314.09,20
1233645,2021-09-09,Lincoln,Wyoming,1978,14.00,56023,WY,County,19830,9974.79,70.60,70.60,9974.79,21
991411,2021-09-09,Sublette,Wyoming,977,11.00,56035,WY,County,9831,9937.95,111.89,111.89,9937.95,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
811581,2021-09-09,Lowndes,Alabama,1651,55.00,1085,AL,County,9726,16975.12,565.49,565.49,16975.12,6,1
1038578,2021-09-09,Hale,Alabama,2786,81.00,1065,AL,County,14651,19015.77,552.86,552.86,19015.77,1,2
875538,2021-09-09,Greene,Alabama,1170,40.00,1063,AL,County,8111,14424.86,493.16,493.16,14424.86,45,3
343978,2021-09-09,Walker,Alabama,9638,300.00,1127,AL,County,63521,15172.93,472.28,472.28,15172.93,32,4
718671,2021-09-09,Dallas,Alabama,4472,171.00,1047,AL,County,37196,12022.80,459.73,459.73,12022.80,61,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546855,2021-09-09,Weston,Wyoming,824,6.00,56045,WY,County,6927,11895.48,86.62,86.62,11895.48,17,19
1202670,2021-09-09,Uinta,Wyoming,3168,16.00,56041,WY,County,20226,15663.01,79.11,79.11,15663.01,4,20
1233645,2021-09-09,Lincoln,Wyoming,1978,14.00,56023,WY,County,19830,9974.79,70.60,70.60,9974.79,21,21
808383,2021-09-09,Albany,Wyoming,5372,23.00,56001,WY,County,38880,13816.87,59.16,59.16,13816.87,9,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.04,6509.08,26,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.04,6509.08,26,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.04,6509.08,26,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.04,6509.08,26,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,80.04,6509.08,26,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623583,2021-09-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1623584,2021-09-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1623585,2021-09-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1623586,2021-09-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
1034270,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19015.77,2,1,1.00
1034271,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19015.77,2,1,0.00
1034272,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19015.77,2,1,0.00
1034273,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,552.86,19015.77,2,1,0.00
1034274,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,552.86,19015.77,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254489,2021-09-05,Crook,Wyoming,574,13.00,56011,WY,County,7584,7568.57,171.41,171.41,8161.92,9,23,0.00
1254490,2021-09-06,Crook,Wyoming,574,13.00,56011,WY,County,7584,7568.57,171.41,171.41,8161.92,9,23,0.00
1254491,2021-09-07,Crook,Wyoming,601,13.00,56011,WY,County,7584,7924.58,171.41,171.41,8161.92,9,23,27.00
1254492,2021-09-08,Crook,Wyoming,611,13.00,56011,WY,County,7584,8056.43,171.41,171.41,8161.92,9,23,10.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
808331,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16975.12,1,6,1.00,0.00
808332,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16975.12,1,6,0.00,0.00
808333,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16975.12,1,6,0.00,0.00
808334,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16975.12,1,6,0.00,0.00
808335,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16975.12,1,6,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340170,2021-09-05,Teton,Wyoming,4502,11.00,56039,WY,County,23464,19186.84,46.88,46.88,19549.10,23,1,0.00,0.00
340171,2021-09-06,Teton,Wyoming,4502,11.00,56039,WY,County,23464,19186.84,46.88,46.88,19549.10,23,1,0.00,0.00
340172,2021-09-07,Teton,Wyoming,4568,11.00,56039,WY,County,23464,19468.12,46.88,46.88,19549.10,23,1,66.00,0.00
340173,2021-09-08,Teton,Wyoming,4572,11.00,56039,WY,County,23464,19485.17,46.88,46.88,19549.10,23,1,4.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-09-09') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
410906,2021-09-09,Imperial,California,33656,750.00,6025,CA,County,181215,18572.41,413.87,413.87,18572.41,1,3,45.00,0.00
2972,2021-09-09,Los Angeles,California,1426515,25563.00,6037,CA,County,10039107,14209.58,254.63,254.63,14209.58,2,7,2186.00,49.00
204084,2021-09-09,San Bernardino,California,340633,5411.00,6071,CA,County,2180085,15624.76,248.20,248.20,15624.76,3,4,369.00,4.00
91009,2021-09-09,Stanislaus,California,77553,1192.00,6099,CA,County,550660,14083.65,216.47,216.47,14083.65,4,9,258.00,7.00
721293,2021-09-09,Inyo,California,1589,39.00,6027,CA,County,18039,8808.69,216.20,216.20,8808.69,5,35,0.00,0.00
72345,2021-09-09,San Joaquin,California,94364,1561.00,6077,CA,County,762148,12381.32,204.82,204.82,12381.32,6,16,259.00,13.00
42042,2021-09-09,Riverside,California,350316,4806.00,6065,CA,County,2470546,14179.70,194.53,194.53,14179.70,7,8,741.00,18.00
653365,2021-09-09,Merced,California,38149,523.00,6047,CA,County,277680,13738.48,188.35,188.35,13738.48,8,10,108.00,1.00
112926,2021-09-09,Tulare,California,69584,877.00,6107,CA,County,466195,14925.94,188.12,188.12,14925.94,9,5,539.00,3.00
40938,2021-09-09,Fresno,California,131266,1831.00,6019,CA,County,999101,13138.41,183.26,183.26,13138.41,10,13,509.00,3.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1535983,2021-09-09,Lassen,California,6501,29.00,6035,CA,County,30573,21263.86,94.85,94.85,21263.86,34,1,0.00,0.00
875480,2021-09-09,Kings,California,28840,274.00,6031,CA,County,152940,18857.07,179.16,179.16,18857.07,11,2,104.00,0.00
410906,2021-09-09,Imperial,California,33656,750.00,6025,CA,County,181215,18572.41,413.87,413.87,18572.41,1,3,45.00,0.00
204084,2021-09-09,San Bernardino,California,340633,5411.00,6071,CA,County,2180085,15624.76,248.20,248.20,15624.76,3,4,369.00,4.00
112926,2021-09-09,Tulare,California,69584,877.00,6107,CA,County,466195,14925.94,188.12,188.12,14925.94,9,5,539.00,3.00
256797,2021-09-09,Kern,California,129808,1491.00,6029,CA,County,900202,14419.87,165.63,165.63,14419.87,14,6,483.00,5.00
2972,2021-09-09,Los Angeles,California,1426515,25563.00,6037,CA,County,10039107,14209.58,254.63,254.63,14209.58,2,7,2186.00,49.00
42042,2021-09-09,Riverside,California,350316,4806.00,6065,CA,County,2470546,14179.70,194.53,194.53,14179.70,7,8,741.00,18.00
91009,2021-09-09,Stanislaus,California,77553,1192.00,6099,CA,County,550660,14083.65,216.47,216.47,14083.65,4,9,258.00,7.00
653365,2021-09-09,Merced,California,38149,523.00,6047,CA,County,277680,13738.48,188.35,188.35,13738.48,8,10,108.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
410906,2021-09-09,Imperial,California,33656,750.00,6025,CA,County,181215,18572.41,413.87,413.87,18572.41,1,3,45.00,0.00
2972,2021-09-09,Los Angeles,California,1426515,25563.00,6037,CA,County,10039107,14209.58,254.63,254.63,14209.58,2,7,2186.00,49.00
204084,2021-09-09,San Bernardino,California,340633,5411.00,6071,CA,County,2180085,15624.76,248.20,248.20,15624.76,3,4,369.00,4.00
91009,2021-09-09,Stanislaus,California,77553,1192.00,6099,CA,County,550660,14083.65,216.47,216.47,14083.65,4,9,258.00,7.00
721293,2021-09-09,Inyo,California,1589,39.00,6027,CA,County,18039,8808.69,216.20,216.20,8808.69,5,35,0.00,0.00
72345,2021-09-09,San Joaquin,California,94364,1561.00,6077,CA,County,762148,12381.32,204.82,204.82,12381.32,6,16,259.00,13.00
42042,2021-09-09,Riverside,California,350316,4806.00,6065,CA,County,2470546,14179.70,194.53,194.53,14179.70,7,8,741.00,18.00
653365,2021-09-09,Merced,California,38149,523.00,6047,CA,County,277680,13738.48,188.35,188.35,13738.48,8,10,108.00,1.00
112926,2021-09-09,Tulare,California,69584,877.00,6107,CA,County,466195,14925.94,188.12,188.12,14925.94,9,5,539.00,3.00
40938,2021-09-09,Fresno,California,131266,1831.00,6019,CA,County,999101,13138.41,183.26,183.26,13138.41,10,13,509.00,3.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5402,09/05/21,Lassen,6444,29.00,21077.42,94.85,34,1,103.00,2.00
5403,09/06/21,Lassen,6444,29.00,21077.42,94.85,34,1,0.00,0.00
5404,09/07/21,Lassen,6444,29.00,21077.42,94.85,34,1,0.00,0.00
5405,09/08/21,Lassen,6501,29.00,21263.86,94.85,34,1,57.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5488,09/05/21,Fresno,129522,1828.00,12963.85,182.96,10,13,525.00,6.00
5489,09/06/21,Fresno,129873,1828.00,12998.99,182.96,10,13,351.00,0.00
5490,09/07/21,Fresno,130227,1828.00,13034.42,182.96,10,13,354.00,0.00
5491,09/08/21,Fresno,130757,1828.00,13087.47,182.96,10,13,530.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)